In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [ ]:
!pip install sociophysicsdatahandler
from sociophysicsDataHandler import SociophysicsDataHandler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# If you have done things correctly, this command should return you your own credentials
!cat auth.txt

In [106]:
# instantiate the storage connection class
dh = SociophysicsDataHandler()
dh.list_files('')

targeting path /storage/surfsara/ProRail_USE_LL_data/
Files listed. Accessible as <this-object>.filelist


,path,name,attributes
0,/storage/surfsara/ProRail_USE_LL_data/asdz/,asdz,"{'{DAV:}getlastmodified': 'Mon, 12 Sep 2022 15..."
1,/storage/surfsara/ProRail_USE_LL_data/config/,config,"{'{DAV:}getlastmodified': 'Mon, 12 Sep 2022 15..."
2,/storage/surfsara/ProRail_USE_LL_data/econophy...,econophysics,"{'{DAV:}getlastmodified': 'Mon, 12 Sep 2022 15..."
3,/storage/surfsara/ProRail_USE_LL_data/ehv/,ehv,"{'{DAV:}getlastmodified': 'Mon, 03 Oct 2022 07..."
4,/storage/surfsara/ProRail_USE_LL_data/sync.sh,sync.sh,"{'{DAV:}getlastmodified': 'Wed, 02 Mar 2022 18..."
5,/storage/surfsara/ProRail_USE_LL_data/upload_l...,upload_log.csv,"{'{DAV:}getlastmodified': 'Wed, 13 Oct 2021 10..."
6,/storage/surfsara/ProRail_USE_LL_data/ut/,ut,"{'{DAV:}getlastmodified': 'Mon, 12 Sep 2022 15..."


In [109]:
# let us look at a specific example
file_path = 'ehv/platform2.1/20210330/'

# show all the files in the above directory
# TODO
fLst = dh.list_files(file_path)
fLst.name[0]

targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330/
Files listed. Accessible as <this-object>.filelist


'EHV_Platform2.1_2021033008_trajectorie.parquet'

In [111]:

# fetch the data from the first file
# TODO
dh.fetch_prorail_data_from_path(file_path + "/" + fLst.name[0])
dh.df['date_time_utc'] = pd.to_datetime(dh.df['date_time_utc'], unit = 'ms')
print(len(dh.df))

trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330//EHV_Platform2.1_2021033008_trajectorie.parquet
data fetched. Accessible as <this-object>.df
359270


In [112]:
df_sorted_by_id = dh.df.sort_values(['tracked_object','date_time_utc'])
unique_ids = df_sorted_by_id.tracked_object.unique()
# print(unique_ids)

print(df_sorted_by_id.head(30))
df_dictionary = {id : df_sorted_by_id[:][df_sorted_by_id.tracked_object==id] for id in unique_ids}
print(df_dictionary[13783])


              date_time_utc  tracked_object  x_pos    y_pos
0   2021-03-30 08:24:46.500           13783   10.0  10854.0
34  2021-03-30 08:24:46.700           13783    0.0  10837.0
69  2021-03-30 08:24:46.800           13783   12.0  10840.0
103 2021-03-30 08:24:46.900           13783   -3.0  10851.0
137 2021-03-30 08:24:47.000           13783  -21.0  10854.0
171 2021-03-30 08:24:47.100           13783  -48.0  10833.0
205 2021-03-30 08:24:47.200           13783  -66.0  10836.0
239 2021-03-30 08:24:47.300           13783  -78.0  10833.0
273 2021-03-30 08:24:47.400           13783  -79.0  10833.0
307 2021-03-30 08:24:47.500           13783  -91.0  10844.0
341 2021-03-30 08:24:47.600           13783  -82.0  10847.0
375 2021-03-30 08:24:47.700           13783  -83.0  10846.0
409 2021-03-30 08:24:47.800           13783  -85.0  10837.0
442 2021-03-30 08:24:47.900           13783 -102.0  10840.0
475 2021-03-30 08:24:48.000           13783 -120.0  10856.0
508 2021-03-30 08:24:48.100           13

In [113]:
averaging_time = 1 #Number of seconds over which we want to average the speed
averaging_frames = int(averaging_time * 10 / 2) #Assuming 10 frames per second
for key in df_dictionary.keys(): 
  
  df_dictionary.get(key).reset_index()
  df_dictionary.get(key)['x_pos_before'] = df_dictionary.get(key)['x_pos'].shift(averaging_frames)
  df_dictionary.get(key)['x_pos_after'] = df_dictionary.get(key)['x_pos'].shift(-averaging_frames)
  df_dictionary.get(key)['y_pos_before'] = df_dictionary.get(key)['y_pos'].shift(averaging_frames)
  df_dictionary.get(key)['y_pos_after'] = df_dictionary.get(key)['y_pos'].shift(-averaging_frames)
  df_dictionary.get(key)['x_speed'] = df_dictionary.get(key)['x_pos_after']-df_dictionary.get(key)['x_pos_before']
  df_dictionary.get(key)['y_speed'] = df_dictionary.get(key)['y_pos_after']-df_dictionary.get(key)['y_pos_before']
  
  df_dictionary.get(key)['speed'] =   ((df_dictionary.get(key)['x_speed'])**2 + (df_dictionary.get(key)['y_speed'])**2)**(1/2)
df_dictionary[13783].head(40)

,date_time_utc,tracked_object,x_pos,y_pos,x_pos_before,x_pos_after,y_pos_before,y_pos_after,x_speed,y_speed,speed
0,2021-03-30 08:24:46.500,13783,10.0,10854.0,NaN,-48.0,NaN,10833.0,NaN,NaN,NaN
34,2021-03-30 08:24:46.700,13783,0.0,10837.0,NaN,-66.0,NaN,10836.0,NaN,NaN,NaN
69,2021-03-30 08:24:46.800,13783,12.0,10840.0,NaN,-78.0,NaN,10833.0,NaN,NaN,NaN
103,2021-03-30 08:24:46.900,13783,-3.0,10851.0,NaN,-79.0,NaN,10833.0,NaN,NaN,NaN
137,2021-03-30 08:24:47.000,13783,-21.0,10854.0,NaN,-91.0,NaN,10844.0,NaN,NaN,NaN
171,2021-03-30 08:24:47.100,13783,-48.0,10833.0,10.0,-82.0,10854.0,10847.0,-92.0,-7.0,92.265923
205,2021-03-30 08:24:47.200,13783,-66.0,10836.0,0.0,-83.0,10837.0,10846.0,-83.0,9.0,83.486526
239,2021-03-30 08:24:47.300,13783,-78.0,10833.0,12.0,-85.0,10840.0,10837.0,-97.0,-3.0,97.046379
273,2021-03-30 08:24:47.400,13783,-79.0,10833.0,-3.0,-102.0,10851.0,10840.0,-99.0,-11.0,99.609238
307,2021-03-30 08:24:47.500,13783,-91.0,10844.0,-21.0,-120.0,10854.0,10856.0,-99.0,2.0,99.020203


In [114]:
# After calculating all speeds of the tracked people, we combine all seperate dataframes back to one dataframe
tracked_objects_list = list(df_dictionary.keys())
#print(tracked_objects_list)

#for i in range(1,len(tracked_objects_list)):
df_with_speeds = pd.concat(df_dictionary)




time_to_investigate = dh.df['date_time_utc'][0]

#df_at_timestamp = dh.df[dh.df['date_time_utc']==time_to_investigate]
#df_at_timestamp.head(20)
#df_at_timestamp.reset_index()
